In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime, timedelta
from glob import glob

In [4]:
files = glob('org.my/*.html')
len(files)

110

In [5]:
all_a = []
for f in tqdm(files):
    with open(f) as fopen:
        data = fopen.read()
    soup = BeautifulSoup(data, "lxml")
    a = soup.find_all('a')
    a = [a_.get('href', '') for a_ in a if a_.get('href', '').endswith('.pdf')]
    all_a.extend(a)

100%|█████████████████████████████████████████| 110/110 [00:07<00:00, 15.04it/s]


In [6]:
all_a = list(set(all_a))
len(all_a)

10092

In [7]:
# !wget https://gist.githubusercontent.com/huseinzol05/98974ae8c6c7a65d4bc0af9f5003786a/raw/5aa5257608b61e8fcc828e99fbd070d5ca7358e3/mp.py

In [8]:
import mp

In [9]:
with open('all_a.json', 'w') as fopen:
    json.dump(all_a, fopen)

In [10]:
all_a_f = [(no, f) for no, f in enumerate(all_a)]

In [12]:
!mkdir pdf

In [13]:
def loop(files):
    files, _ = files
    for f in tqdm(files):
        i, f = f
        file = os.path.join('pdf', f'{i}.pdf')
        if os.path.exists(file):
            continue
        
        try:
            r = requests.get(f, timeout = 60)
            with open(file, 'wb') as fopen:
                fopen.write(r.content)
        except:
            pass

In [14]:
loop((all_a_f[:2], 0))

100%|█████████████████████████████████████████████| 2/2 [00:01<00:00,  1.51it/s]


In [15]:
mp.multiprocessing(all_a_f, loop, cores = 10, returned = False)

100%|███████████████████████████████████████| 1009/1009 [25:45<00:00,  1.53s/it]


In [23]:
!du -hs pdf

16G	pdf
